In [1]:
# Import libraries

import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

In [2]:
# random forest generation returning the mean absolute error

from sklearn.ensemble import RandomForestRegressor

def get_random_forest_mae(X_trn,X_tst,y_trn,y_tst):
  mdl = RandomForestRegressor(random_state=1)
  mdl.fit(X_trn,y_trn)
  y_tst_prd = mdl.predict(X_tst)
  mae = mean_absolute_error(y_tst,y_tst_prd)
  return(mae)

In [5]:
df = pd.read_csv("../austin_311.csv")
df.head()

c:\Users\hamza\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Service Request (SR) Number,SR Description,Method Received,SR Status,SR Location,Street Number,Street Name,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate,(Latitude.Longitude),Council District,Created Date,Closed Date
0,19-00090956,Animal - Proper Care,Phone,Closed,"4609 RIBBECKE AVE, AUSTIN, TX 78721",4609,RIBBECKE,AUSTIN,78721.0,TRAVIS,30.280581,-97.685318,"(30.28058076, -97.68531793)",1.0,2019-03-14 03:02:15,2020-11-23 01:41:21
1,20-00135805,Tree Issue ROW,Mobile Device,Closed,"3521 WEST AVE, AUSTIN, TX 78705",3521,WEST,AUSTIN,78705.0,TRAVIS,30.302857,-97.742312,"(30.30285681, -97.74231249)",9.0,2020-04-07 07:06:32,2020-11-23 12:02:05
2,20-00052942,Pothole Repair,Web,Closed,"7900 E BEN WHITE BLVD WB, AUSTIN, TX",7900,BEN WHITE BLVD WB,AUSTIN,78741.0,TRAVIS,30.219838,-97.688440,"(30.21983764, -97.68844011)",3.0,2020-02-12 01:55:04,2020-02-12 12:57:35
3,19-00467964,ARR Dead Animal Collection,Phone,Closed,"7100 METROPOLIS DR, AUSTIN, TX 78744",7100,METROPOLIS,AUSTIN,78744.0,TRAVIS,30.206036,-97.702592,"(30.20603574, -97.70259216)",2.0,2019-12-10 02:17:04,2019-12-11 09:57:54
4,19-00468033,Austin Code - Request Code Officer,Phone,Closed,"2401 GROVE BLVD, AUSTIN, TX 78741",2401,GROVE,AUSTIN,78741.0,TRAVIS,30.221664,-97.707177,"(30.22166434, -97.70717698)",3.0,2019-12-10 02:51:24,2019-12-10 03:00:05


In [6]:
# define our variables, we want to determine the status so that is our y, all other factors are treated as independent to that event

y = df.loc[:,["SR Status"]]
x = df.drop(["SR Status","SR Location","Created Date", "Closed Date", "(Latitude.Longitude)", "Street Number", "Service Request (SR) Number"], axis = 1)

In [5]:
# selecting numeric features of the dataset
num_columns = [c for c in x.columns if x[c].dtype in ['int64', 'float64']]
x_numeric = x[num_columns]

# using x_numeric.isna() I see there are NaN values in the numeric columns, so I will replace with the mean
x_numeric = x_numeric.fillna(x_numeric.mean())

x_numeric.head()

,Zip Code,Latitude Coordinate,Longitude Coordinate,Council District
0,78721.0,30.280581,-97.685318,1.0
1,78705.0,30.302857,-97.742312,9.0
2,78741.0,30.219838,-97.688440,3.0
3,78744.0,30.206036,-97.702592,2.0
4,78741.0,30.221664,-97.707177,3.0


In [6]:
x_numeric.isna().sum()

Zip Code                0
Latitude Coordinate     0
Longitude Coordinate    0
Council District        0
dtype: int64

In [7]:
cat_obj = [c for c in x.columns if x[c].dtype == 'object' and x[c].nunique() < 144]
x_catergoric = x[cat_obj]
x_catergoric.fillna("hamza", inplace=True)

# x_catergoric.isna().sum()
le = preprocessing.LabelEncoder()

for col in cat_obj:
    x_catergoric[col] = le.fit_transform(x_catergoric[col])

x_catergoric.head()

c:\Users\hamza\anaconda3\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\hamza\AppData\Local\Temp/ipykernel_16096/1750223845.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_catergoric[col] = le.fit_transform(x_catergoric[col])


,SR Description,Method Received,City,County
0,12,12,2,4
1,130,9,2,4
2,92,17,2,4
3,2,12,2,4
4,22,12,2,4


In [8]:
x_catergoric.isna().sum()

SR Description     0
Method Received    0
City               0
County             0
dtype: int64

In [9]:
frames = [x_numeric, x_catergoric]

frames
x_test = pd.concat(frames, axis =1 ,ignore_index=True)

x_test.head()

,0,1,2,3,4,5,6,7
0,78721.0,30.280581,-97.685318,1.0,12,12,2,4
1,78705.0,30.302857,-97.742312,9.0,130,9,2,4
2,78741.0,30.219838,-97.688440,3.0,92,17,2,4
3,78744.0,30.206036,-97.702592,2.0,2,12,2,4
4,78741.0,30.221664,-97.707177,3.0,22,12,2,4


In [10]:
y.head()

,SR Status
0,Closed
1,Closed
2,Closed
3,Closed
4,Closed


In [11]:
y_columns = [c for c in y.columns]

for c in y_columns:
    y[c] = le.fit_transform(y[c])

y.head()

,SR Status
0,1
1,1
2,1
3,1
4,1


In [12]:
x_train_t, x_test_t, y_train_t, y_test_t = train_test_split(x_test, y, test_size=0.2, random_state=1)
get_random_forest_mae(x_train_t, x_test_t, y_train_t, y_test_t)

C:\Users\hamza\AppData\Local\Temp/ipykernel_16096/1589325592.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  mdl.fit(X_trn,y_trn)


0.35447806281047006

In [63]:
# define our variables, we want to determine the status so that is our y, all other factors are treated as independent to that event

date1 = df.loc[:,["Created Date"]]
date2 = df.loc[:,["Closed Date"]]


In [64]:
date1.head()

,Created Date
0,2019-03-14 03:02:15
1,2020-04-07 07:06:32
2,2020-02-12 01:55:04
3,2019-12-10 02:17:04
4,2019-12-10 02:51:24


In [65]:
date2.head()

,Closed Date
0,2020-11-23 01:41:21
1,2020-11-23 12:02:05
2,2020-02-12 12:57:35
3,2019-12-11 09:57:54
4,2019-12-10 03:00:05


In [66]:
def date_list_generator(dateframe):
    date_list = []
    for i in dateframe:
        date_list.append(dateframe[i].replace("-","/")[2:])
    return date_list


In [69]:
date_1_list = date_list_generator(date1)
date_2_list = date_list_generator(date2)
date_1_list

[2         2020-02-12 01:55:04
 3         2019-12-10 02:17:04
 4         2019-12-10 02:51:24
 5         2020-02-12 07:28:04
 6         2019-12-10 03:11:26
                  ...         
 912296    2021-02-11 10:27:26
 912297    2021-02-11 12:17:19
 912298    2021-02-11 12:56:59
 912299    2021-02-11 03:11:08
 912300    2021-02-11 04:18:40
 Name: Created Date, Length: 912299, dtype: object]

In [49]:
for c in date1:
    a = date1[c][0].replace("-","/")[2:]
    b = date1[c][1].replace("-","/")[2:]
print(a)    

19/03/14 03:02:15


In [15]:
b

'20/04/07 07:06:32'

In [16]:
from datetime import datetime


datetime_object_a = datetime.strptime(a, '%y/%m/%d %H:%M:%S')
datetime_object_b = datetime.strptime(b, '%y/%m/%d %H:%M:%S')
# datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')

In [18]:
print(f"Date1: {datetime_object_a} \nDate2: {datetime_object_b}")

Date1: 2019-03-14 03:02:15 
Date2: 2020-04-07 07:06:32


In [19]:
c = datetime_object_b - datetime_object_a

datetime.timedelta(days=390, seconds=14657)